# Data Wrangling

of Project 'Demography behind household sizes in Switzerland and its change in time'

In [281]:
%matplotlib inline
import os
import pandas as pd
import numpy as np

In [132]:
data_folder = './data/'
wage_folder = data_folder+'Gross_monthly_wage_GMW/'

## Datasets

* **hh_sizes:** Distribution of Population in 1-6 person households for 2011 - 2017 || Municipality Level
* **cantonal_parties:** Voter's shares of the political parties in Switzerland for 2007 - 2018 || Cantonal Level
* **demographics_population:** Demographic information (incl. foreigners, age) for 2010 - 2017 || Cantonal Level
* **wage_swiss_foreign:** Mean Income with quantiles and work types for 2008 - 2016 || Regional Level
* **incomes:** Mean Incomes with age-distribution for the year 2007 || Municipality Level
* **rental_13_16:** Rental averages for 2013 - 2016 || Regional Level
* **rental_foreign:** Distribution of rents for 2012 - 2016 || Regional Level
* **rental_sqm_demographics:** Living conditions (demographically) in percents  for 2011 - 2016 || Regional Level

# Load and Prepare Data

In [241]:
#Load raw data and load entire excel sheets
hh_sizes = pd.read_excel(data_folder + 'HH_sizes_commune.xlsx', skiprows=3, skipfooter=10, sheet_name = None, usecols=list(range(9)))
incomes = pd.read_csv(data_folder + 'Incomes.csv')
cantonal_parties = pd.read_excel(data_folder + 'Kantonale Parlamentswahlen.xls', skiprows=3, skipfooter=10, sheet_name = None)
demographics_population = pd.read_excel(data_folder + 'population_regions.xlsx', skiprows=4, skipfooter=10, sheet_name = None)
wage_swiss_foreign_ch = pd.read_excel(wage_folder + 'GMW_switzerland.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_lake_ge = pd.read_excel(wage_folder + 'GMW_lake_geneva.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_mittelland = pd.read_excel(wage_folder + 'GMW_mittelland.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_north = pd.read_excel(wage_folder + 'GMW_north.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_zh = pd.read_excel(wage_folder + 'GMW_zurich.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_east = pd.read_excel(wage_folder + 'GMW_east.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_central = pd.read_excel(wage_folder + 'GMW_central.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
wage_swiss_foreign_ti = pd.read_excel(wage_folder + 'GMW_ticino.xlsx', skiprows=42, skipfooter=10, sheet_name = None, usecols = 10)
rental_11_13 = pd.read_excel(data_folder + 'rental_prices_age_11-13.xls', skiprows=8, skipfooter=10)
rental_13_16_all = pd.read_excel(data_folder + 'rental_prices_age.xlsx', skiprows=5, skipfooter=11,sheet_name = None)
rental_foreign_rooms = pd.read_excel(data_folder + 'rental_prices_nationality.xlsx', skiprows=4, skipfooter=11,sheet_name = None)
rental_rooms_cities_11_13 = pd.read_excel(data_folder + 'rental_prices_room_size_11-13.xls', skiprows=8, skipfooter=12)
rental_rooms_cities_12_14 = pd.read_excel(data_folder + 'rental_prices_room_size_12-14.xls', skiprows=7, skipfooter=10)
rental_sqm_demographics = pd.read_excel(data_folder + 'rental_prices_size.xlsx', skiprows=3, skipfooter=21, sheet_name = None)


In [219]:
len(rental_11_13_rooms.columns)

17

In [220]:
#Functions for the cleaning up of the excel sheets

#rename the headers
def rename_header(df, header):
    for y in df.keys():
        df[y].columns = header
    return df

#drop the NAN values in the sheets
def drop_NA(df):
    for y in df.keys():
        df[y] = df[y].replace(r'^\s*$',np.nan,regex=True)
        df[y] = df[y].dropna(axis= 0,how = 'all')
        df[y] = df[y].dropna(axis= 1,how = 'all')
    return df

#Select the selected rows
def select_X_rows(df,num_rows):
    for y in df.keys():
        df[y] = df[y].iloc[:num_rows]
    return df

#rename the columns
def rename_columns(df, new_names):
    for y in df.keys():
        df[y] = df[y].rename(index=str, columns=new_names)
    return df

#drop the input indexes for a new indexing
def drop_index(df, ind, rows = True):
    
    if rows:
        for y in df.keys():
            df[y] = df[y].drop(ind).reset_index().drop('index', 1)
    else:
        for y in df.keys():
            df[y] = df[y].drop(df[y].columns[ind], axis=1)
    return df

#cleaning of wage regions
def clean_wage_region_data(df):
    header= ["Residents","total_median","quartile_from","quartile_to",
            "women_median","women_quartile_from","woman_quartile_to",
            "men_median","men_quartile_from","man_quartile_to"]
    df = drop_NA(df)
    df = select_X_rows(df,8)
    df = rename_header(df, header)
    return df

#cleaning of rental prices
def clean_rental_csv(df):
    if len(df.columns) > 15:
        drop_cols = [2,4,6,8,10,12,14]
        df = df.drop(df.columns[drop_cols], axis=1)
        header = ["Region","average_rent","1 room","2 rooms",
                    "3 rooms","4 rooms","5 rooms",
                    "6+ rooms"]
        df.columns = header
        df = df.dropna(axis= 0,how = 'all')
    return df

#### Household sizes (from 2011 - 2017) - Most Important Layer - MUNICIPALITIES

In [221]:
col_header = ['Commune', 'Total', '1-person households','2-person households', 
               '3-person households', '4-person households','5-person households', 
               '6-person or larger households', 'not plausible hh']
hh_sizes = rename_header(hh_sizes, col_header)

#### Political Parties (from 2007 - 2018) - CANTONS

In [222]:
cantonal_parties = drop_NA(cantonal_parties)
cantonal_parties = select_X_rows(cantonal_parties,26)
rename_cols = {"Unnamed: 0": "Canton"}
cantonal_parties = rename_columns(cantonal_parties, rename_cols)

#### Demographics Population (from 2010 - 2017) - CANTONS

In [223]:
demographics_population = drop_NA(demographics_population)
rename_cols= {"Unnamed: 0": "Region",
             "Unnamed: 1":"Population",
             "Urban core":"Urban core area",
             "Area influenced":"influenced by urban cores",
             "Area beyond":"influenced beyond urban cores"}
demographics_population = rename_columns(demographics_population, rename_cols)
drop_indices = ['0','1']
if len(demographics_population['2016']) > 35:
    demographics_population = drop_index(demgraphics_population, drop_indices)

#### Mean Wage Foreigners/Swiss (2008/2010/2012/2014/2016) - REGIONS

In [224]:
wage_swiss_foreign_ch = clean_wage_region_data(wage_swiss_foreign_ch)
wage_swiss_foreign_lake_ge = clean_wage_region_data(wage_swiss_foreign_lake_ge)
wage_swiss_foreign_mittelland = clean_wage_region_data(wage_swiss_foreign_mittelland)
wage_swiss_foreign_north = clean_wage_region_data(wage_swiss_foreign_north)
wage_swiss_foreign_zh = clean_wage_region_data(wage_swiss_foreign_zh)
wage_swiss_foreign_east = clean_wage_region_data(wage_swiss_foreign_east)
wage_swiss_foreign_central = clean_wage_region_data(wage_swiss_foreign_central)
wage_swiss_foreign_ti = clean_wage_region_data(wage_swiss_foreign_ti)

**Note :** The wage level with "No management function" was chosen.

#### Rental Prices (from 2011 - 2016) - REGIONS

In [244]:
drop_cols = [2,4,6,8,10,12,14,16]
if len(rental_11_13.columns) > 14:
    rental_11_13 = rental_11_13.drop(rental_11_13.columns[drop_cols], axis=1)
header = ["Region","average_rent","<25","25-64",
            "65+","<25 and 25-64","<25 and 65+",
            "25-64 and 65+","<25, 25-64, and 65+"]
rental_11_13.columns = header
rental_11_13 = rental_11_13.dropna(axis= 0,how = 'all')
drop_rows = ["1 Zimmer","2 Zimmer","3 Zimmer","4 Zimmer", "5 Zimmer", "6 Zimmer und mehr"]

for value in drop_rows:
    rental_11_13 = rental_11_13.drop(rental_11_13.index[rental_11_13["Region"] == value])


In [254]:
drop_cols = [2,4,6,8,10,12,14,16]
if len(rental_13_16_all['2016'].columns) > 14:
    rental_13_16_all = drop_index(rental_13_16_all, drop_cols, rows = False)
rental_13_16 = rename_header(rental_13_16_all, header)

**Legend :** Average rental cost in which specific demographics reside. For instance the column "<25, 25-64, and 65+" means that in this household, resides at least a person that is less than 25 years, a person between 25-64 years, and also a person who is 65 and older. 

#### Rents Distribution Swiss/Foreigner (from 2012 - 2016) - REGIONS

In [168]:
drop_cols = [2,4,6,8]
if len(rental_foreign['2016'].columns) > 6:
    rental_foreign = drop_index(rental_foreign_rooms, drop_cols, rows = False)
header = ["Region","average rent all","household with only Swiss",
            "hh with min 1 Swiss and 1 foreigner","household with only foreigners"]
rental_foreign = rename_header(rental_foreign, header)

drop_rows = ["1 Zimmer","2 Zimmer","3 Zimmer","4 Zimmer", "5 Zimmer", "6 Zimmer und mehr"]
drop_ind = []
for i, value in enumerate(rental_foreign['2016']["Region"]):
    for item in drop_rows:
        if value == item:
            drop_ind.append(i)

rental_foreign = drop_index(rental_foreign, drop_ind)

#### Percentual Living Conditions (from 2011 - 2016) - REGIONS


In [171]:
drop_ind = [2,4,6,8]
if len(rental_sqm_demographics['2016'].columns) > 6:
    rental_sqm_demographics = drop_index(rental_sqm_demographics, drop_ind, rows = False)
rental_sqm_demographics = drop_NA(rental_sqm_demographics)

#Comment: Better cleaning possible with MultiIndexes, can be realized when dataset is used

# Save used data to csv

In [266]:
data_wrangled = './data/wrangled_data/'

In [280]:
#export of the relevant datasets to csvs (cleaned up). Wages are yet missing, as incomes.csv does also include
#the revenues of the people in higher resolution (municipality base). in the export, a .csv file is created
#for each year of the dataset (easier handling afterwards)

for i in hh_sizes:
    hh_sizes[str(i)].to_csv(data_wrangled + 'hh_sizes{}.csv'.format(i))

for i in cantonal_parties:
    cantonal_parties[str(i)].to_csv(data_wrangled + 'elections{}.csv'.format(i))
    
for i in demographics_population:
    demographics_population[str(i)].to_csv(data_wrangled + 'demographics{}.csv'.format(i))
    
for i in rental_13_16:
    rental_13_16[str(i)].to_csv(data_wrangled + 'rentals{}.csv'.format(i))
    
for i in rental_foreign:
    rental_foreign[str(i)].to_csv(data_wrangled + 'rental_foreigners{}.csv'.format(i))
    
for i in rental_sqm_demographics:
    rental_sqm_demographics[str(i)].to_csv(data_wrangled + 'rental_demographics{}.csv'.format(i))
